# Notebook dédiée au test du noyau polynomiale avec les différents classifieurs

## 0.Import des fonctions, créations des fonctions utiles et des classifieurs

In [23]:
import numpy as np 
import pandas as pd 
from scipy.stats import uniform
from scipy.linalg import solve,lstsq
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, ClassifierMixin
from cvxopt import matrix
from cvxopt import solvers
from itertools import product,compress

### 0.1 Kernel Ridge Regression

In [13]:
def Kernel_Ridge_Regression(X_train,y_train,lbd,weight,gamma,degree,c0,k,biais,kernel):
    if kernel=="rbf":
        K=rbf_kernel(X_train,gamma)
    elif kernel=="poly":
        K=poly_kernel(X_train,X_train,degree,c0)
    elif kernel=="spectrum":
        K=spectrum_kernel(X_train,k)
    elif kernel=="precomputed":
        K=X_train
    n=K.shape[0]
    w=weight
    if not(biais):
        if isinstance(weight,bool):
            A=(K+n*lbd*np.eye(n))
            alpha=solve(A,y_train,assume_a="sym")
            return alpha
        elif isinstance(weight,str):
                w1=(y_train==1).mean()
                w0=1-w1
                w=np.where(y_train==1,w1,w0)
        wi=(1/w)
        
        A=K+n*lbd*wi*np.eye(n)
        alpha=solve(A,y_train,assume_a="sym")
     
        return alpha
    else:
        
        Kb=addbiais(K)
        
        K0=addzeros(K)
    
        if isinstance(weight,bool):
            A=(Kb.T.dot(Kb)+lbd*n*K0)
            B=Kb.T.dot(y_train)
            alpha=solve(A,B,assume_a="sym")
            return alpha
        elif isinstance(weight,str):
                w1=(y_train==1).mean()
                w0=1-w1
                w=np.where(y_train==1,w1,w0)
        W=np.diag(w)
        A=(Kb.T.dot(W.dot(Kb))+lbd*n*K0)
        B=Kb.T.dot(W.dot(y_train),assume_a="sym")
        alpha=solve(A,B)
        return alpha
        

In [14]:
class KernelRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False,gamma="auto",degree=2,c0=1,k=3,biais=False,kernel="rbf"):
        self.lbd=lbd
        self.weight=weight
        self.gamma=gamma
        self.degree=degree
        self.c0=c0
        self.k=k
        self.biais=biais
        self.kernel=kernel
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str) and self.kernel=="rbf":
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=Kernel_Ridge_Regression(X,y,self.lbd,self.weight,self.gamma,self.degree,self.c0,self.k,self.biais,self.kernel)
        return self
    def decision_function(self,X):
        if self.kernel=="precomputed":
            return X.dot(self.alpha)
        if not(self.biais):
            if self.kernel=="rbf":
                return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
            elif self.kernel=="poly":
                return poly_kernel(X,self.Xtr,self.degree,self.c0).dot(self.alpha) 
            elif self.kernel=="spectrum":
                return K_spectrum_kernel(X,self.Xtr,self.k).dot(self.alpha) 
        else:
            if self.kernel=="rbf":
                return addbiais(K_rbf_kernel(X,self.Xtr,self.gamma)).dot(self.alpha)
            elif self.kernel=="poly":
                return addbiais(poly_kernel(X,self.Xtr,self.degree,self.c0)).dot(self.alpha)
            elif self.kernel=="spectrum":
                return addbiais(K_spectrum_kernel(X,self.Xtr,self.k)).dot(self.alpha)

    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight,"gamma":self.gamma,"degree":self.degree,"c0":self.c0,"k":self.k,
                "biais":self.biais,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### 0.2 Kernel Logistic Regression

In [15]:
def sigmoid(v):
    return 1/(1+np.exp(-v))
def log_loss(v):
    return np.log(1+np.exp(-v))


In [16]:
def IRLS(X_train,y_train,lbd,ga,degree,c0,k,bs,ker,n_iter,eps=10**-6,method='slow'):
    n=y_train.shape[0]
  
    if ker=="rbf":
        K=rbf_kernel(X_train,ga)
    elif ker=="poly":
        K=poly_kernel(X_train,X_train,degree,c0)
    elif ker=="spectrum":
        K=spectrum_kernel(X_train,k)
    elif ker=="precomputed":
        K=X_train
    #alpha=Kernel_Ridge_Regression(K,y_train,lbd,False,1,bs,"precomputed")
    #alpha=np.zeros(n)
    #l=[]
  
    if bs :
        Kb=addbiais(K)
        K0=addzeros(K)
        alpha=np.zeros(n+1)
    else:
        alpha=np.zeros(n)
    for i in range(n_iter):
   
        alpha_old=alpha
       
        if bs:
            m=Kb.dot(alpha)
            #l.append(log_loss(y_train*m).mean()+lbd*alpha[:-1].dot(K.dot(alpha[:-1])))
        
        else:
            m=K.dot(alpha)
            #l.append(log_loss(y_train*m).mean()+lbd*alpha.dot(m))
        
        
        p=sigmoid(m)
       
        weight=p*(1-p)
       
        weight=np.where(weight<0.000001,0.000001,weight)
       
   
        u=np.where(sigmoid(y_train*m)<0.000001,0.000001,sigmoid(y_train*m))
        z = m + y_train/u
    
        if not(bs):
        
            S = np.diag(weight**-1)
            A=(K+2*lbd*n*S)
            alpha=solve(A,z,assume_a="sym")
            
            #print(np.linalg.norm(alpha_old-alpha))
            
            if np.linalg.norm(alpha_old-alpha)<eps:
                break
        else:
            S = np.diag(weight)
            A=(Kb.T.dot(S.dot(Kb))+2*lbd*n*K0)
            B=Kb.T.dot(S.dot(z))
            if method=="slow":
                alpha=lstsq(A,B)[0]
            else:
                alpha=solve(A,B,assume_a="sym")
            #print(np.linalg.norm(alpha_old-alpha))
            if np.linalg.norm(alpha_old-alpha)<eps:
                break
                
       
    return alpha #,l
        

In [17]:
class KernelLR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,gamma='auto',degree=2,c0=1,k=3,biais=False,kernel="rbf",n_iter=15,method="slow"):
        self.lbd=lbd
        self.gamma=gamma
        self.degree=degree
        self.c0=c0
        self.k=k
        self.biais=biais
        self.kernel=kernel
        self.n_iter=n_iter
        self.method=method
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str) and self.kernel=="rbf":
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=IRLS(X,y,self.lbd,self.gamma,self.degree,self.c0,self.k,self.biais,self.kernel,self.n_iter,method=self.method)
        return self
    def decision_function(self,X):
        if not(self.biais):
            if self.kernel=="precomputed":
                return X.dot(self.alpha)
            if self.kernel=="rbf":
                return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
            elif self.kernel=="poly":
                return poly_kernel(X,self.Xtr,self.degree,self.c0).dot(self.alpha) 
            elif self.kernel=="spectrum":
                return K_spectrum_kernel(X,self.Xtr,self.k).dot(self.alpha) 
        else:
            if self.kernel=="rbf":
                return addbiais(K_rbf_kernel(X,self.Xtr,self.gamma)).dot(self.alpha)
            elif self.kernel=="poly":
                return addbiais(poly_kernel(X,self.Xtr,self.degree,self.c0)).dot(self.alpha)
            elif self.kernel=="spectrum":
                return addbiais(K_spectrum_kernel(X,self.Xtr,self.k)).dot(self.alpha)
        

    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
    def predict_proba_(self,X,y=None):
        p=sigmoid(self.decision_function(X)).reshape(-1,1)
        return hstack((p,1-p))
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"gamma":self.gamma,"degree":self.degree,"c0":self.c0,"k":self.k,"biais":self.biais,
                "kernel":self.kernel,"n_iter":self.n_iter,"method":self.method}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### 0.3 Kernel SVM

In [18]:
def SVM(X_train,y_train,C,gamma,degree,c0,k,kernel):
    n=y_train.shape[0]
    if kernel=="rbf":
        K=rbf_kernel(X_train,gamma)
    elif kernel=="poly":
        K=poly_kernel(X_train,X_train,degree,c0)
    elif kernel=="spectrum":
        K=spectrum_kernel(X_train,k)
    elif kernel=="precomputed":
        K=X_train
    P=matrix(K,tc='d')
    q=matrix(-y_train,tc='d')
    g1=np.diag(y_train)
    G=matrix(np.vstack((g1,-g1)),tc='d')
    h=matrix(np.hstack((np.repeat(C,n),np.zeros(n))),tc='d')
    solvers.options['show_progress'] = False
    sol=solvers.qp(P,q,G,h)
    return np.array(sol['x']).reshape(-1,)

In [19]:
class KernelSVM(BaseEstimator,ClassifierMixin):
    def __init__(self,C=1,gamma='auto',degree="2",c0=1,k=3,kernel="rbf"):
        self.C=C
        self.gamma=gamma
        self.degree=degree
        self.c0=c0
        self.k=k
        self.kernel=kernel
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str) and self.kernel=="rbf":
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=SVM(X,y,self.C,self.gamma,self.degree,self.c0,self.k,self.kernel)
        #idx=self.alpha>10**-5
        #self.Xtr=self.Xtr[idx]
        #self.alpha=self.alpha[idx]
        return self
    def decision_function(self,X):
        if self.kernel=="precomputed":
            return X.dot(self.alpha)
        elif self.kernel=="rbf":
            return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
        elif self.kernel=="poly":
            return poly_kernel(X,self.Xtr,self.degree,self.c0).dot(self.alpha) 
        elif self.kernel=="spectrum":
            return K_spectrum_kernel(X,self.Xtr,self.k).dot(self.alpha) 
            
           
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
    def get_params(self, deep=True):
    
        return {"C": self.C,"gamma":self.gamma,"degree":self.degree,"c0":self.c0,"k":self.k,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### 0.4 Fonctions utiles 

In [ ]:
def présentation_résultat(search,n):
    mask=search.cv_results_['rank_test_score']<=n
    params=list(compress(search.cv_results_['params'], list(mask)))
    mean_test_score=search.cv_results_['mean_test_score'][mask]
    a={}
    for i in range(mean_test_score.size):
        k=''
        for key, value in params[i].items():
            k+=" "+key+" "+str(value)
        a.update({k:mean_test_score[i]})
        sortedDict = sorted(a.items(), key=lambda x: x[1],reverse=True)
    l=[]
    for i in sortedDict:
        u=i[0].split(sep=' ')
        del(u[0])
        lp=[]
        for j in u[1::2]:
            lp.append(j)
        lp.append(i[1])
        l.append(lp)
    head=list(params[0].keys())+["mean_test_score"]

    return(pd.DataFrame(l,columns=head))

In [20]:
def addbiais(X):
    return np.hstack((X,np.ones((X.shape[0],1))))
def addzeros(X):
    n,_=X.shape
    A=np.zeros((n+1,n+1))
    A[:n,:n]=X
    return(A)

In [21]:
def csv_file(models,filename): #models is a list of 3 models
    Y_test=np.empty(0)
    for X_train, Y_train, X_test, model  in zip([X_0train,X_1train,X_2train], [Y_0train,Y_1train,Y_2train], [X_0test,X_1test,X_2test], models):
        model.fit(X_train, Y_train)
        Y_pred=model.predict(X_test)
        Y_test=np.concatenate((Y_test,np.where(Y_pred==-1,0,Y_pred)), axis=0)
    
    Y_test=Y_test.reshape(len(Y_test),1)
    
    ids=np.arange(Y_test.shape[0])
    ids=ids.reshape(len(ids),1)
    
    df=pd.DataFrame(data=np.concatenate((ids,Y_test), axis=1), columns=['Id','Bound'],dtype=np.int)
    
    return df.to_csv('Predictions/'+filename, index = False, header=True)

### 0.5 Kernel polynomiale

In [22]:
def poly_kernel(X,Y,degree,c0):
    return (X.dot(Y.T)+c0)**degree

# 1.Test des différents classifieurs



### 1.0 Import des données

In [4]:
X_0=pd.read_csv("Data/Xtr0_mat100.csv",sep=' ',header=None)
X_1=pd.read_csv("Data/Xtr1_mat100.csv",sep=' ',header=None)
X_2=pd.read_csv("Data/Xtr2_mat100.csv",sep=' ',header=None)


Y_0=pd.read_csv("Data/Ytr0.csv",sep=',')
Y_1=pd.read_csv("Data/Ytr1.csv",sep=',')
Y_2=pd.read_csv("Data/Ytr2.csv",sep=',')

Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])

X_0train=X_0.to_numpy()
X_1train=X_1.to_numpy()
X_2train=X_2.to_numpy()

### 1.1 Tests de Kernel Ridge Regression

In [10]:
KRR=make_pipeline(StandardScaler(),KernelRR(kernel="poly"))
print(KRR)

hps0={'kernelrr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernelrr__degree":[1,2,3,4,5],
      "kernelrr__c0":[0,0.5,1,5,10]}
searchKRR0= GridSearchCV(KRR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR0.fit(X_0train, Y_0train)

hps1={'kernelrr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernelrr__degree":[1,2,3,4,5],
      "kernelrr__c0":[0,0.5,1,5,10]}
searchKRR1= GridSearchCV(KRR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR1.fit(X_1train, Y_1train)


hps2={'kernelrr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernelrr__degree":[1,2,3,4,5],
      "kernelrr__c0":[0,0.5,1,5,10]}
searchKRR2= GridSearchCV(KRR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR2.fit(X_2train, Y_2train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kernelrr', KernelRR(kernel='poly'))])
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
exception calling callback for <Future at 0x2350fd24af0 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\amiri\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 404, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\amiri\anaconda3\lib\multiprocessing\queues.py", line 116, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'Solvers.Kernel_Ridge_Regression'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\amiri\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\amiri\anaconda3\lib\site-packages\joblib\parallel.py", line 366, in __call__
    self.parallel.

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

### 1.2 Tests de Kernel Logistic Regression

In [7]:
KLR=make_pipeline(StandardScaler(),KernelLR(kernel="poly"))
print(KLR)

hps0={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernellr__degree":[1,2,3,4,5],
      "kernellr__c0":[0,0.5,1,5,10]}
searchKLR0= GridSearchCV(KLR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=1)
searchKLR0.fit(X_0train, Y_0train)

hps1={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernellr__degree":[1,2,3,4,5],
      "kernellr__c0":[0,0.5,1,5,10]}
searchKLR1= GridSearchCV(KLR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR1.fit(X_1train, Y_1train)


hps2={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernellr__degree":[1,2,3,4,5],
      "kernellr__c0":[0,0.5,1,5,10]}
searchKLR2= GridSearchCV(KLR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR2.fit(X_2train, Y_2train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kernellr', KernelLR(kernel='poly'))])
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [ ]:
hps0={'kernellr__lbd':uniform(loc=10**-6,scale=10),"kernellr__degree":[4,5],
      "kernellr__c0":[0.5]}
searchKLR0= RandomizedSearchCV(KLR,hps0,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR0.fit(X_0train, Y_0train)

hps1={'kernellr__lbd':uniform(loc=10**-1,scale=10),"kernellr__degree":[2],"kernellr__c0":[5,10]}
searchKLR1= RandomizedSearchCV(KLR,hps1,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR1.fit(X_1train, Y_1train)


hps2={'kernellr__lbd':uniform(loc=10**-6,scale=10),"kernellr__degree":[3,4],
      "kernellr__c0":[5,10]}
searchKLR2= RandomizedSearchCV(KLR,hps2,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR2.fit(X_2train, Y_2train)

### 1.3 Test de Kernel SVM

# 3 Elaboration du model final et création du fichier csv